# 本项目使用说明

由于本次比赛，[水印擦除挑战赛](https://aistudio.baidu.com/aistudio/competition/detail/209/0/introduction)评分提交的是模型加模型参数文件，因此就没有再加入对A，B榜的图片进行推理的notebook。因此本项目只关心于如何训练模型。

首先运行下面cell的代码，对数据集进行解压。

In [ ]:
# 解压文件
!unzip -oq data/data145795/train_dataset.zip -d ./dataset
!unzip -oq data/data145795/valid_dataset.zip -d ./dataset

## 1、数据处理

> 数据处理的方式决定了模型的设计，也会对预测的精度产生较大的影响。与手写文字擦除任务一个比较大的区别是：水印占据面积很大，因此对水印擦除后，还需要对被擦除的区域进行一个填补，这个是该项目的难点所在。为了显示的引导模型进行预测，需要结合gt和img做差值来生成mask。如下图，从左向右依次为img,gt,mask(用自己的代码生成的，参考generate_mask.py，代码中图片路径供参考，是在本地电脑进行处理的):
<div align=center><img src='imgs/gt_img_mask.png'></div>
<center>图1.1 加了水印的图像(左)，真实图像(中)，mask(右)</center>

> 另一方面，由于本次比赛数据集过大，1841张本体图像，每张本体图生成551张带水印的图像，一共1841x551张，100多G。其实到后面就会发现，这个任务模型推理出mask的位置是比较简单的，因为mask是十分规律的，但是生成依然做的不够好，所以要扩充数据集最好是找到1841张本体图像的分布然后进行扩充。

> 虽然机器学习定理告诉我们，训练数据量越多模型效果越好，越不容易过拟合；但这是有前提的，因为我们无法做到全批量梯度下降，真实的训练过程我们只会一次一个小batch的训练，最早期的batch对模型的梯度影响必然会被后期的batch洗掉一部分，反向传播决定了模型不能进行增量学习。所以，在显存不大的情况下，过大训练数据集起到的作用得不偿失，将数据集控制在20G之内既加快了项目打开的速度，也不会掉精度。

> 参考手写文字擦除，我们同样将图片进行裁剪(随机裁剪至512, 512大小)，对密集预测型任务不使用resize。

总结一下：在数据处理部分，我们一共使用了三种策略，
1、缩减数据集100G-->10G
2、生成mask引导模型训练
3、随机裁剪至512x512大小

## 2、模型搭建

A榜用的是[Erasenet](https://ieeexplore.ieee.org/stamp/stamp.jsp?tp=&arnumber=9180003)，模型代码参考了https://aistudio.baidu.com/aistudio/projectdetail/3439691 ， 同手写文字擦除一样，我们更改了loss函数，因为这个方式比较直接效果显著(模型是需要训练的，并不是设计的越复杂越好，直接调整面向真实数据的loss设计可以有效改变模型训练的轨迹)。模型结构图如下：
<div align=center><img src='imgs/Erasenet.png'></div>
<center>图2.1 Erasenet主体结构</center>

模型数据流向大体如上，loss的地方做了一定的修改。

B榜对模型进行了一次调优，方法是将网络最开始下采样和精修部分下采样的卷积替换成了[SwinT模块](https://aistudio.baidu.com/aistudio/projectdetail/3288357)，就像在我之前Swin那个项目里一样，将Swin和CNN成功的结合起来，做到又快又好，最终B榜分数也比较高。下图展示了原Erasenet和**带swin的Erasenet**改在验证集上的表现，psnr分别是31.418，33.042。
<div align=center><img src='imgs/img_erase_erase改_gt.png'></div>
<center>图2.2 从左到右依次为img, 原始erasenet，erasenet改，gt</center>
再次印证了SwinT单个模块的强大力量！

## 3、训练模型

> ***运行trainstr.ipynb可以训练原始erasenet***，训练日志log和最好的模型都已包含在项目中，用visualdl即可可视化。虽然最后不会用这个模型提交，但还是放在这，可以起到一个参考的作用，因为batchsize达到28，所以训练起来是要比erasenet改快一点的。

> erasenet改是分两部分训练的，***开始是用的A100，运行trainswin.ipynb即可***，但是A100只能训练24小时，因此***将最好的模型加载再使用V100进行训练，运行trainswinv100即可***。log_swin，log_swin_v100包含了完整的训练日志。我们只是因为时间紧迫才用的A100训练的，但这并不是必要的，单纯用V100多训练几天也是可以的。

## 4 模型预测

模型预测部分的代码保存在predict.py文件中，同在训练过程中对模型进行评估的处理方法是一致的，预测为mask的地方取模型的输出，预测为非mask的地方取输入图片的像素。这样在非mask的地方就可以保证像素差接近0（因为jpg图像本身有一些噪点，一般达不到0）。